# Glue Connect to snowflake

1. **Add Snowflake JARs to Glue**:
- Download: snowflake-jdbc-3.13.30.jar and spark-snowflake_2.12-2.11.3-spark_3.3.jar
- Upload to S3: s3://your-bucket/jars/

2. **Glue Job Setup**:
```python
from awsglue.context import GlueContext
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .config("spark.jars", "s3://your-bucket/jars/spark-snowflake_2.12-2.11.3-spark_3.3.jar,s3://your-bucket/jars/snowflake-jdbc-3.13.30.jar") \
    .getOrCreate()

glueContext = GlueContext(spark.sparkContext)
```

3. **Snowflake Connection Configuration**:
```python
sf_options = {
    "sfURL": "your-account.snowflakecomputing.com",
    "sfUser": "glue_user",
    "sfPassword": "your_password",
    "sfDatabase": "NURSING_HOME",
    "sfSchema": "PUBLIC",
    "sfWarehouse": "GLUE_WH"
}
```


4. **Read/Write Example**:
```python
# Read from Snowflake
df = spark.read.format("snowflake") \
    .options(**sf_options) \
    .option("query", "SELECT * PROVIDERS") \
    .load()

# Write to Snowflake
processed_data.write.format("snowflake") \
    .options(**sf_options) \
    .option("dbtable", "STAFFING_REPORTS") \
    .mode("overwrite") \
    .save()
```

# Key Considerations
1. **IAM Integration**:

- Prefer Snowflake Key Pair Authentication over passwords

- Use AWS Secrets Manager for credentials
```python
secret = glueContext.get_secret(secretId="snowflake-creds")
sf_options["sfPassword"] = secret["password"]
```

2. **Performance Optimization**:
```python
.option("sfCompress", "on") \
.option("parallelism", "8") \
.option("autopushdown", "on")
```

3. **Error Handling**:
```python
.option("continue_on_error", "on") \
.option("load_error_log", "@ERROR_LOG")
```

4. Data Type Mappings:

- Use explicit casts for Snowflake semi-structured data (VARIANT, ARRAY)
- Handle TIMESTAMP_NTZ/TIMESTAMP_LTZ explicitly 


# Deployment Process

#### Cloudformation Steps:
```python
   cloudformation/
    ├── 1-iam-roles-stack.json       # IAM roles/policies
    ├── 2-s3-buckets-stack.json      # S3 bucket configurations
    ├── 3-glue-snowflake-stack.json  # Glue resources + Snowflake integration
    ├── 4-glue-workflow-stack.json
    └── parameters/
        ├── dev-params.json          # Environment-specific parameters
        └── prod-params.json
    ├── 5-sns-sqs-notification-stack.json  # notify using outlook.com
    ├── 6-s3-template-notify.json  # policy for s3 notification sns
        
```

1. **Deploy IAM stack first**:
```python
   aws cloudformation deploy \
    --template-file 1-iam-roles-stack.json \
    --stack-name iam-stack \
    --parameter-overrides EnvironmentType=dev \
    --capabilities CAPABILITY_NAMED_IAM
```

2. **Deploy S3 stack**:
```python
aws cloudformation deploy \
    --template-file 2-s3-buckets-stack.json \
    --stack-name nursing-home-dev-s3 \
    --parameter-overrides EnvironmentType=dev
```

3. **Deployment Strategy**

- **Upload Dependent Scripts**:
   ```python
      # Create local directory
    mkdir -p health_citations_etl
    
    # Move script to directory
    mv health_citations_etl.py health_citations_etl/
    
    # Now copy recursively
    aws s3 cp health_citations_etl/ s3://nh-source-657082399901-dev/scripts/ --recursive
   ```

- **Verify Upload:**
 - aws s3 ls s3://nh-source-657082399901-dev/scripts/ --recursive

4. **Deploy Glue stack**:
```python
aws cloudformation deploy \
    --template-file 3-glue-snowflake-stack.json \
    --stack-name nursing-home-dev-glue \
    --parameter-overrides \
        EnvironmentType=dev \
        SnowflakeSecretARN=arn:aws:secretsmanager:us-east-1:657082399901:secret:connection_parameters_snowflake_dev-Tc7c3a \
        SnowflakeAccount=xxxxxx1.us-east-1 \
    --capabilities CAPABILITY_NAMED_IAM
```

- #### Make changes to the to `Policy` section: `Secret`
    - **secret name**: connection_parameters_snowflake
    - **secret arn**: arn:aws:secretsmanager:us-east-1:657082399901:secret:connection_parameters_snowflake-EqHWQS

5. **Load files from `Google Drive` to `S3` Bucket**

```python
python3 drive_to_s3_sync.py 
```

6. **Deploy Workflow Stack**:
```python
aws cloudformation deploy \
  --template-file 4-glue-workflow-stack.json \
  --stack-name nursing-home-dev-workflow \
  --parameter-overrides \
      EnvironmentType=dev \
      SourceBucket=nh-source-657082399901-dev \
  --capabilities CAPABILITY_NAMED_IAM
```

- ***Describe the stack just change stack name***
 - aws cloudformation describe-stacks --stack-name glue-stack


7. **Confirm that secret is configure correctly**
   
   ```python
   aws secretsmanager get-secret-value --secret-id connection_parameters_snowflake
   ```

8. **Create the Database in AWS CLI**
```python
aws glue create-database --database-input '{"Name": "nh_raw_dev", "Description": "Raw data for Nursing Home ETL"}'

```

9. **Start Glue Crawler**
```python
aws glue start-crawler --name dev-raw-data-crawler
```

## Summary

```python

aws cloudformation deploy \
    --template-file 1-iam-roles-stack.json \
    --stack-name nursing-home-dev-iam \
    --parameter-overrides EnvironmentType=dev SnowflakeSecretARN=arn:aws:secretsmanager:us-east-1:657082399901:secret:connection_parameters_snowflake_dev \
    --capabilities CAPABILITY_NAMED_IAM

aws cloudformation deploy \
    --template-file 2-s3-buckets-stack.json \
    --stack-name nursing-home-dev-s3 \
    --parameter-overrides EnvironmentType=dev

aws cloudformation deploy \
    --template-file 3-glue-snowflake-stack.json \
    --stack-name nursing-home-dev-glue \
    --parameter-overrides \
        EnvironmentType=dev \
        SnowflakeSecretARN=arn:aws:secretsmanager:us-east-1:657082399901:secret:connection_parameters_snowflake_dev \
        SnowflakeAccount=xxxxxx1.us-east-1 \
    --capabilities CAPABILITY_NAMED_IAM
    

aws cloudformation deploy \
  --template-file 4-glue-workflow-stack.json \
  --stack-name nursing-home-dev-workflow \
  --parameter-overrides \
      EnvironmentType=dev \
      SourceBucket=nh-source-657082399901-dev \
  --capabilities CAPABILITY_NAMED_IAM

aws cloudformation deploy \
  --template-file 5-notifications-stack.json \
  --stack-name nursing-home-notifications-stack \
  --parameter-overrides EnvironmentType=dev NotificationEmail=sjjb78@hotmail.com \
  --capabilities CAPABILITY_NAMED_IAM
```

# Final Database Structure (Snowflake-Ready)

We'll use:

- **Database**: NURSING_HOME_DEV
- **Schemas**:
    - SOURCE → Raw loaded and cleaned tables
    - MART → Aggregated/analytical tables (Quality, Cost, Facility, Operational, etc.)
    - LOOKUP → Lookup/reference tables (e.g., provider)

## 📂 NURSING_HOME_DEV.SOURCE Tables (based on your actual files)

| Table Name     | Description                                   | Primary Key | Foreign Key Relationships                                                                                                                                     |
|----------------|-----------------------------------------------|-------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| PROVIDER_INFO  | Facility-level details                        | ccn         | -                                                                                                                                                                                                     |
| HEALTH_CITATION| Survey citations                              | composite: ccn, deficiency_tag, survey_date | ccn → PROVIDER_INFO.ccn<br>deficiency_tag → CITATION_DESCRIPTION.deficiency_tag                                                                                                                      |
| CITATION_DESCRIPTION | Tag descriptions                        | deficiency_tag | -                                                                                                                                                                                                     |
| SURVEY_SUMMARY | High-level survey summaries                   | survey_summary_id (generated) | ccn                                                                                                                                                                                                   |
| PENALTIES      | Penalties and fines per facility              | penalty_id (generated) | ccn                                                                                                                                                                                                   |
| STAFFING_METRICS (optional) | Staffing-level data (optional CSV or source) | staffing_id (generated) | ccn                                                                                                                                                                                                   |


## 📂  NURSING_HOME_DEV.MART Tables (Metric Marts — Analytical Layer)
These will be created via **dbt or Snowflake views** from SOURCE layer.

| Table | Description | Based On |
|-------|-------------|----------|
| MART.STAFFING | Aggregated staffing metrics | SOURCE.STAFFING_METRICS |
| MART.FACILITY | Facility metrics (occupancy, bed usage) | SOURCE.PROVIDER_INFO, SOURCE.SURVEY_SUMMARY |
| MART.QUALITY | Quality & satisfaction KPIs | SOURCE.HEALTH_CITATION, SOURCE.CITATION_DESCRIPTION, external survey |
| MART.COST | Cost analysis (payroll etc.) | SOURCE.STAFFING_METRICS, SOURCE.PENALTIES |
| MART.OPERATIONAL | Ops metrics (shifts, staffing load) | SOURCE.STAFFING_METRICS, SOURCE.SURVEY_SUMMARY |


In [ ]:
## 📐 Schema Design Example for STAFFING_METRICS (Optional File or Table)